# USW-AI-Projekt: Einfluss sozialer Medien auf Aktienkurse
## CRISP-DM Struktur
---

### 1. Business Understanding
- Ziel: Zusammenhang zwischen Twitter-Stimmung und Aktienkursentwicklung untersuchen
- Zielgruppe: Finanzanalysten, Anleger
- Fragestellung: Kann man Kursbewegungen durch Stimmung erklären oder sogar vorhersagen?

### 2. Data Understanding
- Datenquellen: Twitter API v2, yfinance
- Zeitraum: z. B. letzte 3–6 Monate
- Unternehmen: z. B. SAP, Siemens, BMW

- Daten vorbereiten & alle nötigen Libaries laden


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as stat

%matplotlib inline
sns.set()

# To avoid Warning message inbetween ...
import warnings
warnings.filterwarnings('ignore')

### 3. Data Preparation
- Dataset laden
- Hier werden die .csv-Dateien geladen:
- Tweets.csv enthält Tweets über Tesla (z. B. Datum, Text, Sentiment)

In [39]:
import pandas as pd

company_tweet = pd.read_csv("../Datasets/raw/tweets_top_companies/Company_Tweet.csv")
tweet         = pd.read_csv("../Datasets/raw/tweets_top_companies/Tweet.csv")
company       = pd.read_csv("../Datasets/raw/tweets_top_companies/Company.csv")
company_value = pd.read_csv("../Datasets/raw/company_values/CompanyValues.csv")


### 4. Merge Data
- Daten zusammenführen
- Jetzt sind für jeden Tag Kursdaten + Durchschnittssentiment in einer gemeinsamen Tabelle

In [42]:
# das sind jetzt die Daten von 2015 bis 2020, weil es davor keine Daten für Tweets gibt
# 1. Datumsspalten korrekt konvertieren
company_value['day_date'] = pd.to_datetime(company_value['day_date'])
tweet['post_date'] = pd.to_datetime(tweet['post_date'], unit='s', errors='coerce')

# 2. Tweets außerhalb gültiger Zeitspanne entfernen (NaT etc.)
tweet = tweet.dropna(subset=['post_date'])

# 3. Sentiment berechnen mit VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
tweet['sentiment_score'] = tweet['body'].apply(lambda x: analyzer.polarity_scores(str(x))['compound'])

# 4. Durchschnittliches Sentiment pro Tag berechnen
daily_sentiment = (
    tweet
    .assign(Date = tweet['post_date'].dt.normalize())  # nur das Datum
    .groupby('Date')['sentiment_score']
    .mean()
    .reset_index()
)

# 5. Kursdaten auf Zeitraum der Tweets beschränken
start = tweet['post_date'].min()
end   = tweet['post_date'].max()

company_value = company_value[
    (company_value['day_date'] >= start) & (company_value['day_date'] <= end)
]

# 6. Merge durchführen (gemeinsames Datum)
merged_df = pd.merge(
    company_value,
    daily_sentiment,
    left_on='day_date',
    right_on='Date',
    how='inner'
)

# 7. Ergebnis anzeigen
print("Merged Shape:", merged_df.shape)
merged_df.head()


Merged Shape: (10950, 9)


,ticker_symbol,day_date,close_value,volume,open_value,high_value,low_value,Date,sentiment_score
0,AAPL,2019-12-31,293.65,25247630,289.93,293.68,289.5200,2019-12-31,0.149347
1,AAPL,2019-12-30,291.52,36059610,289.46,292.69,285.2200,2019-12-30,0.139221
2,AAPL,2019-12-27,289.80,36592940,291.12,293.97,288.1200,2019-12-27,0.154059
3,AAPL,2019-12-26,289.91,23334000,284.82,289.98,284.7000,2019-12-26,0.169727
4,AAPL,2019-12-24,284.27,12119710,284.69,284.89,282.9197,2019-12-24,0.161671


### 5. Korrelation berechnen
- Korrelation zeigt, ob positive Tweets mit steigenden Kursen einhergehen (+1 = stark positiv)

In [26]:
print(merged_df[['sentiment_score', 'close_value']].isnull().sum())


sentiment_score    0
close_value        0
dtype: int64


In [15]:
correlation = merged_df[['sentiment_score', 'close_value']].corr()
print("📊 Korrelation zwischen Sentiment und Schlusskurs:")
print(correlation)

📊 Korrelation zwischen Sentiment und Schlusskurs:
                 sentiment_score  close_value
sentiment_score              NaN          NaN
close_value                  NaN          NaN


### 6.Visualisierung

Die Entwicklung des Aktienkurses von Tesla (Close-Preis)
Den durchschnittlichen Sentiment-Wert pro Tag (z. B. aus Twitter-Daten)

In [ ]:
import matplotlib.pyplot as plt

# x-Achse = Datum, y-Achse = Schlusskurs (Close) der Tesla-Aktie, durchschnittliches Sentiment pro Tag (multipliziert mit 100, um auf ähnlicher Skala wie Aktienkurs sichtbar zu sein)
plt.figure(figsize=(12,5)) # Diagramm mit einer Größe von 12 x 5
plt.plot(merged_df['Date'], merged_df['Close'], label='Schlusskurs', color='blue')
plt.plot(merged_df['Date'], merged_df['Avg_Sentiment'] * 100, label='Sentiment (x100)', color='orange')

plt.legend()
plt.title("Tesla: Sentiment vs. Aktienkurs")
plt.xlabel("Datum")
plt.ylabel("Wert")
plt.grid(True)
plt.show()


📌 Zeigt die Verteilung der Sentiment-Scores (z. B. wie viele Tweets sind stark negativ, positiv oder neutral).

In [ ]:
column_distplot(tweet_df, column_name="afinn_score", bins=10)
column_distplot(tweet_df, column_name="vander_score", bins=10)


🔹 2. Liniendiagramm: Sentiment vs. Aktienkurs über Zeit

📌 Visualisiert zeitlich: Wann starke positive oder negative Tweets auftraten und wie sich der Aktienkurs parallel dazu veränderte.



In [ ]:
def sentiment_overtime(tweet_df, stock_df, title, score_column_name="score"):
    fig = plt.figure(figsize=(24,10))
    ax1 = fig.add_subplot()
    ax2 = ax1.twinx()

    # Sentiment (Vader oder Afinn)
    ax1.vlines(tweet_df['day_date'], 0, tweet_df[score_column_name])
    ax1.axhline(y=0, color='r', linestyle='-')

    # Aktienkurs (Close)
    ax2.plot(stock_df['day_date'], stock_df['close_value'], color='orange', label='Stock price')

    ax2.set_title("Effects of " + title +" tweets to stock price")
    ax1.set_xlabel('Datum')
    ax1.set_ylabel('Sentiment Score', color="blue")
    plt.show()
